# Chapter 30 
# Hierarchical Bayesian Models

## Introduction     

In this chapter extend Bayesian methods to the class of hierarchical models. Up until now, we have focused on pooled models, which pool all data when estimating model parameters. However, as we have seen already, many real-world datasets have discrete categories or strata. For example, a chemist might measure a reaction rate at several discrete levels of pH and temperature, or the problem might involve demographic categories or different geographic regions.    

A pooled model simply computes model coefficients that give the best fit to the overall dataset. You can think of the pooled model as having a flat structure, in the sense that all model coefficients are at the same level. The pooling maximizes statistical power by using all available data to fit a model but clearly lacks flexibility if the classes in the data have different behaviors. 

Another extreme alternative is to use an unpooled model. For each category a separate and independent unpooled model is fit. While this approach maximizes flexibility, the variance of each of the, possibly many, models will be larger. As with the pooled model you can think of unpooled models as having a flat structure.       

The idea of hierarchical models is to find the best of both worlds with an approach that is between the extremes of pooled and unpooled models. Hierarchical models are constructed by creating a hierarchy of hyperpriors. The Hyperpriors represent the prior information on the pooled data. The hyperpriors act as priors for the specific cases. In this way, the hierarchical models occupy a middle ground between pooled and unpooled models.      

In this chapter, we focus on three key points:   
1.	Pooled vs. unpooled models.  
2.	Defining hierarchical models through hyperpriors and priors.   
3.	Evaluation and comparison of Bayesian models.   


## Danger of Radon in Homes   

The unstable isotope Radon-222 is an invisible and orderless gas that is the product of a nuclear decay chain of Uranium 238. The decay chain that creates Radon-222 is shown in the figure below.    


<img src="../images/UtoRn.png" alt="Drawing" style="width:400px; height:600px"/>
<center> Decay chain of Uranium-238 to Radon-222 <center> 
<center> Credit: <a href="https://en.wikipedia.org/wiki/Radon-222">Wilipedia Radon-222 article!</a> <center>
    
Depending on rock and soil type, trace amounts of Uranium-238 are present. Depending on the chemistry, Uranium is [effectively transported by groundwater](https://www.nrc.gov/docs/ML0931/ML093160829.pdf). Hence the decay products, including Radon-222, are dispersed in the environment. Under certain 
    
The health risk of Radon infiltration into homes is [well-documented](https://www.epa.gov/radon/health-risk-radon). Radon is the most prevalent cause of lung cancer among US nonsmokers. In the outdoors, Radon-222 does not accumulate in dangerous quantities. However, within poorly ventilated buildings, the Radon gas cannot dissipate and can accumulate to dangerous levels. Since the Radon molecule is heavy, it tends to accumulate in low areas, such as basements. Further, houses connected to wells or in contact with groundwater are more likely to accumulate dangerous levels of Radon. The diagram below illustrates these points.      

<img src="../images/Radon_Home_Diagram.png" alt="Drawing" style="width:550px; height:400px"/>
<center> Common ways Radon-222 enters homes <center> 
<center> Credit: <a href="https://matracking.ehs.state.ma.us/Environmental-Data/radon/radon_lessons.html">Massachusetts Department of Public Health!</a> <center>

As has already been mentioned the danger from Randon changes with soil and rock composition as well as groundwater chemistry. These facts mean that the risk changes significantly with geographic location. An example the map below shows the Radon risk for the counties of Iowa and Minnesota. Notice that the risk varies considerably in space.      

<img src="../images/MN_IA_CountryRadonMap.jpg" alt="Drawing" style="width:500px; height:600px"/>
<center> Map of Radon in homes for counties in IA and MN <center> 
<center> Credit: <a href="http://employees.csbsju.edu/dsteck/mnradon/">Minnesota Radon Project!</a> <center>

## Bayesian Modeling of Radon Concentration      

In this chapter, we will use the prediction of radon concentration in counties of Minnesota as a running example for exploring and comparing hierarchical Bayesian models. We will construct and compare three models:    
1. A pooled model with a single intercept and slope for all counties.    
2. An unpooled model with separate intercepts and slopes for each county.   
3. A hierarchical model with hyper-priors for the behavior of all counties and at the next level the slopes and intercepts for each county computed using the hyper-priors.   

***********
Note: This notebook is based in part on [this example](https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-hierarchical.html) from the PyMC documentation.   
***********

To get started execute the code in the cell below to import the packages required for the examples. 

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr
import scipy.stats as spst

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
az.style.use("arviz-darkgrid")

Next, execute the code in the cell below to load the dataset and display the head of the data frame.    

In [ ]:
radon_data = pd.read_csv(pm.get_data("radon.csv"))
county_names = radon_data.county.unique()
radon_data.head()

The response variable is the log of radon concentration. The independent variable is the `floor` indicator for the presence of a basement.

In order to construct the PyMC models, we need to create a dictionary with the counties and county names and indices. Execute the code in the cell below to create the dictionary and display a sample of the county index numbers.     

In [ ]:
county_idxs, counties = pd.factorize(radon_data.county)
coords = {
    "county": counties,
    "obs_id": np.arange(len(county_idxs)),
}
print('county_idxs = ' + str(county_idxs[:100]))

Notice that there are several samples for each county.    

## Pooled Model

The first model we will examine is a **pooled model**. The pooled model is a linear regression model of the log of Radon concentration using single slope and intercept coefficients. The coefficient values are computed from the data pooled over all of the counties, hence the name. You can think of this model as flat, with respect to county. In other words, we assume the observations within each county are **[exchangeable](https://en.wikipedia.org/wiki/Exchangeable_random_variables)** with each other. Exchangeable values are assumed to be iid and therefore have the same variance.      

Typically for a regression model the pooled model uses a Normal likelihood model:    

\begin{align}
log(radon) &\sim \mathtt{N}(\mu_c, \epsilon)\\ 
where\
\mu_c &= \beta x_c  
\end{align}

The value of $\mu$ is computed **deterministically** using the model coefficient vector, $\beta$, and the vector of independent variable values, $x$. 

The structure of the model is flat with just parameters $[\beta, \epsilon]$. These parameters have prior distributions:     

\begin{align}   
\beta &\sim \mathtt{N}(0, \sigma)\\ 
\epsilon &\sim |\mathtt{Cauchy}(\eta)\\
\end{align}   

Where, $|\mathtt{Cauchy}$ is the half Cauchy distribution with one parameter, $\eta$. This distribution is often used for variance priors since it has heavy tails and no values $< 0$. To get a feel for this distribution execute the code in the cell below.       

In [ ]:
plt.style.use('arviz-darkgrid')
x = np.linspace(0, 5, 200)
for b in [0.5, 1.0, 2.0, 5.0]:
    pdf = spst.cauchy.pdf(x, scale=b)
    plt.plot(x, pdf, label=r'$\eta$ = {}'.format(b))
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.legend(loc=1)
plt.show()

Notice that the value of the half Cauchy is at a maximum of 0. As the value of $\eta$ increases, the dispersion of the distribution increases.     

We can repersent this **flat Bayesian model** as a directed acyclic graph (DAG), as shown below. The single likelihood model for all the observations is at the bottom level. The prior distributions for the parameters are at the top level.     

<img src="../images/FlatBayesDAG.PNG" alt="Drawing" style="width:600px; height:200px"/>
<center> DAG of Flat Bayesian Model <center> 

### Defining and sampling the model

The PyMC code for the model is shown in the cell below. Read the comments to understand the code. Execute this code to instantiate the model object.   

In [ ]:
with pm.Model() as pooled_model:
    # Independent variable is just number of floors   
    floor = pm.Data("floor", radon_data.floor.values, mutable=True)   
    # Priors for unknown model parameters
    betas = pm.Normal("betas", mu=0, sigma=100, shape=2)
    sigma = pm.HalfCauchy("sigma", 5)

    # Deterministic expected value of outcome
    radon_est = betas[0] + betas[1] * floor

    # Likelihood (sampling distribution) of observations
    y = pm.Normal("y", mu=radon_est, sigma=sigma, observed=radon_data.log_radon) 

### Prior predictive checks

With the pooled model defined, we can perform prior predictive checks on the model. The code in the cell below performs the prior predictive sampling of the model and displays the density of the Betas.  Execute this code and examine the results. 

In [ ]:
SEED = 3434
with pooled_model:
    pooled_prior_checks = pm.sample_prior_predictive(samples=500, random_seed=SEED)

_,ax = plt.subplots(1,2,figsize=(8,3))
for i in range(2): 
    sns.kdeplot(pooled_prior_checks['prior'].betas[0,:,i], ax=ax[0]);
    ax[0].set_title('Density of Betas');   
    
sns.kdeplot(pooled_prior_checks['prior'].sigma[0,:], ax=ax[1]);
ax[1].set_title('Density of Sigma');    

The density of the Beta priors looks reasonable, but with a wide dispursion.   

You can now MCMC sample the model by executing the code in the cell below.  

In [ ]:
with pooled_model:
    pooled_trace = pm.sample(2000, return_inferencedata=True)

The quality of the sampling needs to be verified. To get started, execute the code in the cell below to display the trance plots, by executing the code in the cell below.    

In [ ]:
az.plot_trace(pooled_trace, combined=False, var_names=["betas", "sigma"]); 

Examine these plots. The traces and the density plots look nearly the same for the different traces. Further, the dispersion is greatly srunk compared to the prior predictive densities. This is promising, indicating good convergence of the MCMC sampling.        

The next step is to check the ESS for the parameters. Execute the code in the cell below to display the sampling summary table.    

In [ ]:
az.summary(pooled_trace, kind='diagnostics')

Examine the summary table. All of these performance statistics look reasonable, given the number of samples for the trances          

### Inference on the model parameters       

With the MCMC sampling completed, inference can now be performed on the model parameters. Execute the code in the cell below to display the forest plot for the model parameters.    

In [ ]:
az.plot_forest(pooled_trace, var_names=["betas", "sigma"]);

We can make some simple inferences about the parameters of this model.    
1. The HDI of $\beta_0$ the intercept and $\sigma$ are in a relatively narrow range.   
2. The HDI of the slope parameter, $\beta_1$, is considerably larger than the other parameters. This indicates that the effect size for the home having a basement or not, is poorly determined.   

### Posterior predictive checks    

Finally, we must perform some posterior predictive checks. We must verify that realizations of predicted values from the posterior distribution agree with the distribution of the actual observations. To do so, execute the code in the cell below and examine the results.    

In [ ]:
SEED = 6545
with pooled_model:
    pm.sample_posterior_predictive(pooled_trace, extend_inferencedata=True, random_seed=SEED)
_,ax = plt.subplots(3,1, figsize=(8,9))
az.plot_ppc(pooled_trace, ax=ax[0]);
az.plot_bpv(pooled_trace, kind='p_value', ax=ax[1]);
az.plot_bpv(pooled_trace, kind='u_value', ax=ax[2]);

Examine the plots, noting the following:    
1. The agreement between the observed and the realizations simulated from the posterior distribution of the response variable is generally good in the middle of the distribution. There is one deviation in that the peak of the observed values is just to the right of the maximum of the realized values. Further, the agreement between the posterior and observed shows a deviation in both tails.     
2. The distribution of the Bayesian p-values shows a skew with respect to the ideal distribution. This indicates some systematic error in the posterior distribution.     
3. The Bayesian u-values deviate a bit from the ideal 1.0, but stay within the desired credible interval, indicating any bias in the model is not significant.    

## Unpooled model    

We will now move on to the unpooled model. The unpooled model treats each county independently, with separate coefficients. In more technical terms, the county is the unit of **exchangeability** for the unpooled model. Any county can be exchanged with a common variance. The unpooled model is at the other extreme when compared to the pooled model which has a single set of model parameters with the observations as the unit of exchangeability. 

For the unpooled model, we will use the subscript $c$ to indicate county. The likelihood for the unpooled model can be written:    

\begin{align}   
log(radon_c) &\sim \mathtt{N}(\mu_c, \epsilon)\\ 
where\
\mu_c &= \beta_c x_c  
\end{align}

The value of $\mu_c$ is computed **deterministically** using the model coefficient vector, $\beta_c$, and the vector of independent variable values, $x_c$. For unpooled models, there are separate vectors $\beta_c$ and $x_c$ for each county.      

The structure of the model for a county is flat with parameters $[\beta_c, \epsilon]$. These parameters have prior distributions:     

\begin{align}   
\beta_c &\sim \mathtt{N}(0, \sigma)\\ 
\epsilon &\sim |\mathtt{Cauchy}(\eta)\\
\end{align}   

We use the same priors for all counties.      

As with the pooled Bayes model we can represent the unpooled Bayes model as a DAG, as shown below. The difference here is that the model has a different location parameter, $\mu_C$, for each county, $C$. This difference is why the model is considered **unpooled**. The same prior distributions of the parameters are used for each county.   

<img src="../images/UnpooledBayesDAG.png" alt="Drawing" style="width:600px; height:200px"/>
<center> DAG of Unpooled Bayesian Model <center> 
    
> **Plate notation:** The rectangle shown around the likelihood is known as a **plate**. The plate is used to represent the fact that there are, in fact, multiple distributions. In this case, the plate indicates that there is one likelihood for each county, $C$.      

### Define and sample the model     

The code in the cell below defines the unpooled PyMC model. Notice that the model parameters now have dimensions of the number of counties, except for the error term, $\epsilon$. The likelihood is now vector (multiple) valued on a county-by-county basis. More details can be seen in the code and comments. Execute the code in the cell to instantiate the model.                

In [ ]:
with pm.Model(coords=coords) as unpooled_model:

    # Independent parameters for each county
    county_idx = pm.Data("county_idx", county_idxs, dims="obs_id")
    floor = pm.Data("floor", radon_data.floor.values, dims="obs_id", mutable=True)

    Beta0 = pm.Normal("Beta0", 0, sigma=100, dims="county")
    Beta1 = pm.Normal("Beta1", 0, sigma=100, dims="county")

    # Model error
    eps = pm.HalfCauchy("eps", 5)

    # Model prediction of radon level
    # BetaX[county_idx] translates to a[0, 0, 0, 1, 1, ...],
    # we thus link multiple household measures of a county
    # to its coefficients.
    radon_est = Beta0[county_idx] + Beta1[county_idx] * floor

    # Data likelihood
    y = pm.Normal("y", radon_est, sigma=eps, observed=radon_data.log_radon, dims="obs_id")

### Prior predictive checks

With the model defined we can perform some prior predictive checks. Tje process is nearly the same as for the pooled model There are now coeficients for each county. We will only display plots for the first 20. Execute the code in the cell below and examine the result.   

In [ ]:
SEED = 3434
with unpooled_model:
    unpooled_prior_checks = pm.sample_prior_predictive(samples=500, random_seed=SEED)

_,ax = plt.subplots(2,2,figsize=(10,4))
ax = ax.flatten()

for i in range(20): 
    sns.kdeplot(unpooled_prior_checks['prior'].Beta0[0,:,i], ax=ax[0]);
    ax[0].set_title('Density of Beta0');    
for i in range(20): 
    sns.kdeplot(unpooled_prior_checks['prior'].Beta1[0,:,i], ax=ax[1]);
    ax[1].set_title('Density of Beta1');    
    
sns.kdeplot(unpooled_prior_checks['prior'].eps[0,:], ax=ax[2]);
ax[2].set_title('Density of eps');    

THe densities are very similar to those of the pooled model. There is little difference in the parameters a or b between counties.   

The code in the cell below samples the PyMC model. Given the higher dimensionality of the problem (larger number of parameters) we use 2,000 MCMC samples after the burn-in period. Execute this code.      

In [ ]:
with unpooled_model:
    unpooled_trace = pm.sample(2000, return_inferencedata=True)

The next step is to evaluate the trace plots. Execute the code in the cell below. 

In [ ]:
az.plot_trace(unpooled_trace, var_names=['Beta0','Beta1','eps'], combined=False);

There is a posterior distribution of each of the intercept and slope parameters for each of the counties, 85 of each. There is only one deviation parameter, $\epsilon$ common to all the counties. Keeping the structure of the model in mind, examine these plots noting the following:   
1. From the density plots of the intercept, $a$, it is clear that there is a considerable difference between the distributions for the counties. There appears to be good agreement  With $n_chains \times 85$ traces the plots are hard to interpret. In general, the trace plots show no abnormalities.       
2. The range of the slope parameter values, $b$, is quite wide. Most of the values are in a narrow range but with some distributions with a wide range.  Again, the trace plot is hard to interpret but shows no particular problem.   
3. The density plots for the dispersion parameter, $\epsilon$, show a reasonable range of values and are similar between the traces. The trace plot shows no particular problem, with good agreement between the traces.   
4. The wide range of intercept and slope parameter values is consistent with an unpooled model. For some counties, only a few observations are used to estimate these parameters.   

Next, execute the code in the cell below to see a summary table of the sampling statistics.   

In [ ]:
## To see all rows of the table, uncomment the line of code below and execute. 
az.summary(unpooled_trace, kind='diagnostics')

Examine this table and notice the following:   
1. The MCSE and bulk ESS values are reasonable for the intercept parameter, $a$. As might be expected, the tail ESS values are significantly less than the Bulk ESS, but not so much as to cause problems.   
2. The MCSE values for the slope parameter, $b$, vary over two orders of magnitude. This is consistent with the wide range of slope values seen in the posterior density plots. The bulk ESS values are reasonable. As might be expected, the tail ESS values are significantly less than the Bulk ESS, as a result of the wide tails on some of the posterior distributions.   
3. The MSCE for the dispersion parameter $\epsilon$ is quite small. The bulk ESS and tail ESS are lower than for the other parameters, but not so low as to cause serious problems.   

### Posterior predictive checks    

Finally, we will perform some posterior predictive checks on the unpooled model. Execute the code in the cell below to display the graphs.  

In [ ]:
SEED = 6567
with unpooled_model:
    pm.sample_posterior_predictive(unpooled_trace, extend_inferencedata=True, random_seed=SEED)
_,ax = plt.subplots(3,1, figsize=(8,9))
az.plot_ppc(unpooled_trace, ax=ax[0]);
az.plot_bpv(unpooled_trace, kind='p_value', ax=ax[1]);
az.plot_bpv(unpooled_trace, kind='u_value', ax=ax[2]);

> **Exercise 30-1:** Examine the plots and compare them to the plots for the pooled model. Answer the following quesitons:     
> 1. Compare the posterior predictive mean to the observed distribution of the pooled and unpooled models?         
> 2. Copare the Nayesian p-value curves between the pooled and unpooled models?      
> 3. Wjat can you observe about the Bayesian U-value of the unpooled model and what does this tell you about the fit of that model?     

> **Answers:**    
> 1.                    
> 2.              
> 3.            

## Hierarchical Model

We will now turn our attention to a hierarchical model. The hierarchical model is an intermediate between a pooled model and an unpooled model. As a result, hierarchical models are sometimes called **partial pooled models**.   

Working from the top of the hierarchy downward, the construction of the model requires the following:      
1. Definition of **hyperprior distributions** at the top level of the hierarchy. The hyperpriors can be thought of as overall priors for all parameters, regardless of category. In this case, we have one set of hyperpriors for all counties for the intercept and slope parameters. The use of hyperpriors is the key difference between the partially pooled hierarchical model and the unpooled model.       
2. Prior distributions are defined for each parameter for each category. These priors use the hyperpriors as their prior distributions. The dimensionality of these priors is the same as for the unpooled model. In contrast, a pooled model has only a single prior for each parameter.           
3. A multivalued (vector-valued) likelihood model, using the priors, is defined. 

At the bottom level of the hierarchy, a likelihood model is defined for each county:    

\begin{align}   
log(radon_c) &\sim \mathtt{N}(\mu_c, \epsilon)\\ 
where\
\mu_c &= \beta_c x_c  
\end{align}

The value of $\mu_c$ is computed **deterministically** using the model coefficient vector, $\beta_c$, and the vector of independent variable values, $x_c$. For the hierarchical models, there are separate vectors $\beta_c$ and $x_c$ for each county.      

The structure of the model for a county is flat with parameters $[\beta_c, \epsilon]$. These parameters have prior distributions:     

\begin{align}   
\beta_c &\sim \mathtt{N}(\beta_h, \epsilon_h)\\ 
\epsilon &\sim |\mathtt{Cauchy}(\eta)\\
\end{align}   

Notice that the prior for the dispersion parameter $\epsilon$ is the same for all counties. As with the pooled model.        

Finally, at the top level of the hierarchy, we define the hyperpriors used to estimate $\beta_c$. These hyperpriors are the same for all cases, all counties.    

\begin{align}    
\beta_h &\sim \mathtt{N}(0, \sigma)\\ 
\epsilon_h &\sim |\mathtt{N}(0, \eta)\\
\end{align}    

The hierarchical structure of the model can be illustrated by a DAG, as shown below. At the root of the graph is the likelihood of the model. The plate notation shows that there is a separate likelihood model for each country. The priors of the likelihood model are shown at the next level up. The plat notation shows that there is a separate prior for each of the countries. The **hyperpriors** for the parameters of the priors are shown at the top level. There is only one hyperprior distribution for each parameter.   

<img src="../images/HeirarchicalBayesDAG.png" alt="Drawing" style="width:600px; height:350px"/>
<center> DAG of Hierarchicall Bayesian Model <center> 

Execute the code in the cell below to instantiate this model. 

In [ ]:
with pm.Model(coords=coords) as hierarchical_model:
    county_idx = pm.Data("county_idx", county_idxs, dims="obs_id", mutable=True)
    # Hyperpriors for group nodes
    mu_0 = pm.Normal("mu_0", mu=0.0, sigma=20)
    sigma_0 = pm.HalfNormal("sigma_0", 5.0)
    mu_1 = pm.Normal("mu_1", mu=0.0, sigma=20)
    sigma_1 = pm.HalfNormal("sigma_1", 5.0)

    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    Beta0 = pm.Normal("Beta0", mu=mu_0, sigma=sigma_0, dims="county")
    # effect difference between basement and floor level
    Beta1 = pm.Normal("Beta1", mu=mu_1, sigma=sigma_1, dims="county")

    # Model error
    eps = pm.HalfCauchy("eps", 5.0)

    radon_est = Beta0[county_idx] + Beta1[county_idx] * radon_data.floor.values

    # Data likelihood
    radon_like = pm.Normal("radon_like", mu=radon_est, sigma=eps, observed=radon_data.log_radon, dims="obs_id")

Next, execute the code in the cell below to perform MCMC sampling from the model.  

In [ ]:
with hierarchical_model:
    hierarchical_trace = pm.sample(4000, tune=4000, target_accept=0.95, return_inferencedata=True)

Notice the warning messages produced indicating there is a problem with the hierarchical model. The acceptance rate has already been raised to 0.95 and the number of samples for burn-in and posterior sampling set to 4000. It is likely that we need a different model.   

Execute the code in the cell below to display the trace plot to further explore the problems with this model.

In [ ]:
az.plot_trace(hierarchical_trace, var_names=['mu_0', 'sigma_0', 'mu_1', 'sigma_1', 'Beta0','Beta1','eps'], combined=False);

Examine the density plots. Notice the divergences of the $\sigma_b$ densities. These divergences indicate a problem with the prior distribution of the dispersions.        

### Defining an improved model    

There are several observations we can make to help us define a better model.    
1. The posterior predictive analysis for both the pooled and unpooled models shows the likely presence of outliers in the observations. A sensible and commonly applied approach to creating robust Bayesian models is to use a heavy-tailed distribution, such as the Student T distribution. The heavy-tailed distribution allows for outliers in the sampling of the posterior distributions.        
2. The Cauchy distribution used for the dispersion parameter is perhaps not ideal, since the maximum is at 0. From even simple exploration of the data, it is clear the dispersion of the posterior distributions is far from 0. A commonly used alternative is the inverse gamma distribution. Unlike the Cauchy and Half Normal, the inverse gamma distribution is 0 at 0. Like the Cauchy, the inverse Gamma has a long right tail.   

To display the general shape of the Student T distribution for several parameter values, execute the code in the cell below.   

In [ ]:
#plt.style.use('seaborn-darkgrid')
x = np.linspace(-8, 8, 200)
mus = [0., 0., 0., 0.]
sigmas = [1., 1., 5., 5.]
dfs = [1., 10., 1., 10.]
for mu, sigma, df in zip(mus, sigmas, dfs):
    pdf = spst.t.pdf(x, df, loc=mu, scale=sigma)
    plt.plot(x, pdf, label=r'$\mu$ = {}, $\sigma$ = {}, $\nu$ = {}'.format(mu, sigma, df))
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.legend(loc=1)
plt.show()

Examine this plot. The dispersion of the Student T distribution increases with the value of $\sigma$ and decreases with the degrees of freedom, $\nu$. To create a robust model with will use a prior with larger $\sigma$ and smaller $\nu$.       

To gain some feel for the shape of the inverse Gamma distribution of several parameter choices execute the code in the cell below.   

In [ ]:
x = np.linspace(0, 3, 500)
alphas = [1., 2., 1., 2.]
betas = [2., 2., 1., 1.]
for a, b in zip(alphas, betas):
    pdf = spst.invgamma.pdf(x, a, scale=b)
    plt.plot(x, pdf, label=r'$\alpha$ = {}, $\beta$ = {}'.format(a, b))
plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.legend(loc=1)
plt.show()

Notice how the shape of the inverse gamma distribution changes with the parameters. The model shifts right with larger values of $\alpha$. The right tail becomes heavier with larger values of $$\beta. 

With these new choices for hyperpriors and the response in mind, we can update our definition of the hierarchical model. At the bottom level of the hierarchy, we use a Student T likelihood model. The Student T distribution better accounts for the outliers in the response variable.      

\begin{align}   
log(radon_c) &\sim StudentT(\mu_c, \epsilon, \nu)\\ 
where\\
\mu_c &= \beta_c x_c  
\end{align}

As before, the value of $\mu_c$ is computed **deterministically** using the model coefficient vector, $\beta_c$, and the vector of independent variable values, $x_c$. For the hierarchical models, there are separate vectors $\beta_c$ and $x_c$ for each county.      

The structure of the model for a county is flat with parameters $[\beta_c, \epsilon]$. These parameters have the same prior distributions as before:     

\begin{align}   
\beta_c &\sim \mathtt{N}(\beta_h, \epsilon_h)\\ 
\epsilon &\sim |\mathtt{Cauchy}(\eta)\\
\end{align}   

Notice that the prior for the dispersion parameter $\epsilon$ is the same for all counties. As with the pooled model.        

Finally, at the top level of the hierarchy, we define new hyperpriors to estimate $\beta_c$. These hyperparameters differentiate this model from the first hierarchical model. These hyperpriors are the same for all cases, all counties.    

\begin{align}    
\beta_h &\sim StudentT(0, \sigma, \nu)\\ 
\epsilon_h &\sim InverseGamma(0, \eta, \kappa)\\
\end{align}    

We can represent this improved hierarchical model with the DAG shown below.   

<img src="../images/HeirarchicalBayesDAG2.png" alt="Drawing" style="width:600px; height:350px"/>
<center> DAG of Improved Hierarchicall Bayesian Model <center> 

Execute the code in the cell below to instantiate the new hierarchical model.    

In [ ]:
with pm.Model(coords=coords) as hierarchical_model:
    county_idx = pm.Data("county_idx", county_idxs, dims="obs_id", mutable=True)
    # Hyperpriors for group nodes
    mu_0 = pm.StudentT("mu_0", mu=0.0, sigma=20.0, nu=5)
    sigma_0 = pm.InverseGamma("sigma_0",alpha=1.0, beta=2.0)
    mu_1 = pm.StudentT("mu_1", mu=0.0, sigma=20.0, nu=5)
    sigma_1 = pm.InverseGamma("sigma_1", alpha=1.0, beta=2.0)

    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    Beta0 = pm.Normal("Beta0", mu=mu_0, sigma=sigma_0, dims="county")
    # effect difference between basement and floor level
    Beta1 = pm.Normal("Beta1", mu=mu_1, sigma=sigma_1, dims="county")

    # Model error
    eps = pm.HalfCauchy("eps", 5.0)

    radon_est = Beta0[county_idx] + Beta1[county_idx] * radon_data.floor.values

    # Data likelihood
    radon_like = pm.StudentT("radon_like", mu=radon_est, sigma=eps, nu=5, observed=radon_data.log_radon, dims="obs_id")

> **Exercise 30-2:** Examine the structure of the hierarchical model and answer the following questions.      
> 1. How is the unpooled model different from the pooled model?
> 2. Considering the principle of exchangeability, what is a key similarity between the unpooled and hierarchical models?        
> 3. How do the hyperparameters relate to the observations across counties?         
> 4. How do the number of priors and number of counties relate to each other?
> 5. How can you relate the principle of exchangeability to the set of likelihoods?   

> **Answers:**      
> 1.       
> 2. 
> 3.        
> 4.               
> 5.               

### Prior predictive checks      

Often with hierarchical models, the interactions of the hyperpriors may not be intuitive. Therefoe, before going any further we should check the hyperprior distributions for the new hierarchical model.   

Execute the code in the cell below to sample the prior distributions.    

In [ ]:
SEED = 3434
with hierarchical_model:
    hierarchical_prior_checks = pm.sample_prior_predictive(samples=100, random_seed=SEED)
hierarchical_prior_checks.keys()

With the prior predictive sampling complete, execute the code in the cell below to display the hyperprior densities.  

In [ ]:
_,ax=plt.subplots(3,2,figsize=(10,6))
ax=ax.flatten()
for i,var in enumerate(zip([hierarchical_prior_checks['prior'].Beta0.to_numpy().flatten(), 
                        hierarchical_prior_checks['prior'].Beta1.to_numpy().flatten(),
                        hierarchical_prior_checks['prior'].mu_0.to_numpy().flatten(), 
                        hierarchical_prior_checks['prior'].mu_1.to_numpy().flatten(),
                        hierarchical_prior_checks['prior'].sigma_0.to_numpy().flatten(), 
                        hierarchical_prior_checks['prior'].sigma_1.to_numpy().flatten()],
                        ['Beta0','Beta1','mu_0','mu_1','sigma_0','sigma_1'])):
    sns.kdeplot(var[0], ax=ax[i]);
    ax[i].set_title(var[1])

The hyperprior distributions, `a` and `b`, look reasonable given what we know so far from our analysis of this problem. The range of values seems sensible, and are sufficient to not overly constrain the solution. The parameter priors look wide enough to not restrict the solution as well. Note that `sigma_a` and `sigma_b` will not have values less than 0, but the KDE kernel does not cut-off at 0.      

### MCMC sampling       

With the model defined and the hyperpriors investigated we are ready to MCMC sample the model. Execute the code in the cell below to do so.  

In [ ]:
with hierarchical_model:
    hierarchical_trace = pm.sample(4000, tune=4000, target_accept=0.95, return_inferencedata=True)

With the new model defined the divergence warnings have been eliminated. Still, the sampling is not ideal. You can see the warning about low ESS. With an ESS of about 400 for some parameters, the sampling may still be adequate if not ideal.     

To further investigate the sampling, execute the code in the cell below to display the trace plots.   

In [ ]:
az.plot_trace(hierarchical_trace, var_names=['mu_0', 'sigma_0', 'mu_1', 'sigma_1', 'Beta0','Beta1','eps'], combined=False);

Compare these trace plots to the previous hierarchical model. You can see that the traces and densities for $\sigma_b$ now converge. Further, you can see that the posterior density of the slope parameters, $b$, are more varied for the different counties. Overall, these trace plots show a reasonable sampling of the model parameters.      

Next, execute the code in the cell below to display the summary table of sampling statistics.

In [ ]:
az.summary(hierarchical_trace, kind='diagnostics')

Overall the MCSE and ESS values are reasonable. The exceptions are the bulk and tail ESS of the hyperparamers, particularly $b$ and $\sigma_b$. Still, overall the ESS looks sufficient, if not ideal.  

### Inference on the model       

With a reasonable sampling of the hierarchical model, we are in a position to perform some inference on the model. Execute the code in the cell below to display the forest plot of the model parameters.  

In [ ]:
az.plot_forest(hierarchical_trace, var_names=['mu_0', 'sigma_0', 'mu_1', 'sigma_1','Beta0','Beta1','eps']);

> **Exercise 30-3:** Examine the forest plot and answer the following questions:     
> 1. Examine the HDIs of the county intercept parameters. Using the HDIs or credible intervals, can yuou identify cases of statistically significant differences?
> 2. Can you identify statistically significant differences in the slope coefficients?     
> 3. What do your foregoing inferences tell you about the sparial (county) changes in mean and slope (basement) effects in this model.          

> **Answers:**
> 1.                  
> 2.             
> 3.           

### Posterior predictive checks    

Finally, execute the code in the cell below to sample the posterior predictive distributions and display the charts.  

In [ ]:
SEED = 6565
with hierarchical_model:
    pm.sample_posterior_predictive(hierarchical_trace, extend_inferencedata=True, random_seed=SEED) 

_,ax = plt.subplots(3,1, figsize=(8,9))    
az.plot_ppc(hierarchical_trace, num_pp_samples=100, ax=ax[0]);    
az.plot_bpv(hierarchical_trace, kind='p_value', ax=ax[1]);
az.plot_bpv(hierarchical_trace, kind='u_value', ax=ax[2]);

Examine these plots noticing the following:      
1. The realizations of the response variables are in reasonable agreement with the observations, with a few noticeable deviations. As a result of using the Student T distribution for the response, the tails of the realized distribution are noticeably heavier and more consistent with the observations.        
2. The Bayesian p-values are closer to the ideal distribution than either the pooled or unpooled models.      
3. There is the deviation of the Bayesian u-values from ideal behavior in the tails of the posterior distribution. These deviations are reduced with respect to the unpooled model.   

In summary, the posterior predictive distribution of the hierarchical model is an improvement over the pooled or unpooled model.  

## Comparing Models   

We have now constructed and independently evaluated three Bayesian models. The question now is how can we compare the performance of these models. The method used is based on a **leave one out (LOO)** **cross-validation (CV)** algorithm. This method is a variation on the jackknife resampling method briefly introduced in Chapter 14.          

Specifically, the **expected log pointwise density (ELPD)** is a measure of deviance (see Chapter 22). The ELPD cannot be computed directly since we do not know the true probability density of the response variable, $p(y)$. Instead, we use an approximation using the likelihood and the posterior density. For $n$ observations $y_i$ and model parameters, $\theta$, we can write the ELPD:           

$$ELPD_{LOO_CV} = \sum_{i=1}^n log \int p(y_i\ |\ y_{-i}, \theta)\ p(\theta\ |\ y_{-i}) d \theta$$   

Here:   
$y = $ the set of observations.
$y_{-i} = $ the set of observations less the ith.     
$p(y_i\ |\ y_{-1}, \theta) = $ the likelihood of the ith observation, computed with the ith observation help back.    
$p(\theta\ |\ y_{-i}) = $ the posterior distribution computed excluding the ith observation. 

At first, this relationship does not seem intuitive. To gain some understanding consider the terms inside the integral. The first term is the likelihood of the held-back observation from the distribution comptued without the held back observation. This likelihood is weighted by the posterior probability of the model parameters, given the data used to fit the model, with the observation held back. The integral is the density at the held back observation of the posterior computed without the held back observation. 

$$p(y_i |\  y_{-i})= \int p(y_i\ |\ y_{-i}, \theta)\ p(\theta\ |\ y_{-i}) d \theta$$

These densities are summed over the observations. So in summary, we can say that $ELPD_{LOO_CV}$ is sum of the densities with an observation held back.     

The computational complexity of directly resampling the observations and recomputing the model would be prohibitive. Instead, an importance sampling algorithm known as Pareto smoothed importance sampling (PSIS) is used. The PSIS algorithm resamples from the already computed traces, making the computation of ELPD quite efficient.   

You cabn find additional details on the interpretaton and compuation of $ELPD_{LOO_CV}$ in [this Vinette in the Stan documentation](https://mc-stan.org/loo/articles/loo2-non-factorized.html).  

There are a number of steps required to compute the $ELPD_{LOO_CV}$ for the three models. 

As a first step, we must compute the leave-one-out log-likelihood of each model we wish to compare. The code in the cell does this by these steps:   
1. The log-likelihood of each model is computed from the sampled traces.    
2. The leave-on-out samples of the log-likelihood are then computed.    

Execute the code in the cell below.    

> **Note:** The unpooled model is an insufficient sample, which will generate warning messages.   

In [ ]:
for mod, trace in zip([hierarchical_model,pooled_model,unpooled_model],[hierarchical_trace,pooled_trace,unpooled_trace]):
    with mod:
        pm.compute_log_likelihood(trace)

for loo, trace in zip(['hierarchical_loo','pooled_loo','unpooled_loo'],[hierarchical_trace,pooled_trace,unpooled_trace]):
    loo = az.loo(trace)

With the leave one out log-likelihood of each model computed the models can now be compared. Execute the code in the cell below to create and display the model comparison table.  

In [ ]:
df_comp_loo = az.compare({"hierarchical":hierarchical_trace, "pooled":pooled_trace, 'unpooled':unpooled_trace})
df_comp_loo

The table is ordered by the rank of the model, with the best model shown as $0$. The other columns in the table are:       
- *elpd_loo* is the $ELPD_{LOO_CV}$ of the model. Less negative values are better.   
- *p_loo* is a pseudo estimate of the model complexity, an approximation of the number of parameters.   
- *elpd_diff* is the difference in $ELPD_{LOO_CV}$ between the model and the best model.      
- *weight* are weights used for model averaging.      
- *se* is the standard error of the $ELPD_{LOO_CV}$.     
- *dse* is the standard error of the elpd_diff.       
- *warning* a warning that the $ELPD_{LOO_CV}$ estimate may not be reliable.    
- *scale* the scale used to report $ELPD_{LOO_CV}$.   

You can display a graphical representation of the $ELPD_{LOO_CV}$ and differences by executing the code in the cell below.   

In [ ]:
_,ax = plt.subplots(figsize=(8,4))
az.plot_compare(df_comp_loo, insample_dev=False, ax=ax);

> **Exercise 30-4:** Examine the foregoing table and plot and answer the following quesitons:
> 1. What the the relationship of the $ELPD_{LOO_CV}$ for the three models tell you about the fit to the data?     
> 2. Is the relationship you discussed above for the best model significant and why?    
> 3. Is the difference bwtween the other two models significantly different and why?      

> **Answers:**         
> 1.             
> 2.            
> 3.              

## Bayesian Shrinkage   

We have already seen there is a significant difference between the hierarchical model and the other models. By imposing a hierarchy of priors on the model the values of the county intercept and slope are said to **shrink** toward the prior. In other words, using a hyperprior constrains the model fit and prevents poorly determined parameters from taking extreme values.  

We have already seen evidence of parameter shrinkage in the hierarchical model from the fairly consistent HDIs of the model parameters. We will now explore the shrinkage of the parameters further.    

As a first step, we must compute the MAP of the unpooled and hierarchical model parameters. Execute the code in the cell below to perform these operations.    

In [ ]:
with unpooled_model:
    unpooled_MAP = pm.find_MAP()
with hierarchical_model:
    hierarchical_MAP = pm.find_MAP()    

With MAP values of the parameters computed, we can examine the distribution of these parameters for the two models. To display density plots of the two model parameters execute the code in the cell below.

In [ ]:
_,_ax_ = plt.subplots(2,1,figsize=(10,7))
for var,ax in zip(['Beta0','Beta1'],_ax_):
    sns.kdeplot(unpooled_MAP[var], ax=ax, label='Unpooled')
    sns.kdeplot(hierarchical_MAP[var], ax=ax, label='Hierarchical')
    ax.set_title('Distributions of parameter ' + var + ' values')
    ax.legend()

Examine the foregoing plots. Notice that the density of the unpooled model parameters is much wider than that of the hierarchical model parameters. In other words, the hierarchical model parameter values are in a fairly narrow range compared to the unpooled model. This effect is an example of **Bayesian shrinkage** of the model parameters!    

We can gain another perspective on Bayesian shrinkage by examining how the model parameters for each case, or county in this case, change between the unpooled and hierarchical models. To display this relationship, execute the code in the cell below.  

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(
    111,
    xlabel="Parameter Beta0",
    ylabel="Parameter Beta1",
    title="Hierarchical vs. Unpooled Parameter MAP Values",
    xlim=(0, 3),
    ylim=(-3, 3),
)

ax.scatter(unpooled_MAP["Beta0"], unpooled_MAP["Beta1"], s=25, alpha=0.4, label="Unpooled")
ax.scatter(hierarchical_MAP["Beta0"], hierarchical_MAP["Beta1"], c="red", s=25, alpha=0.4, label="Hierarchical")
for i in range(len(unpooled_MAP["Beta0"])):
    ax.arrow(
        unpooled_MAP["Beta0"][i],
        unpooled_MAP["Beta1"][i],
        hierarchical_MAP["Beta0"][i] - unpooled_MAP["Beta0"][i],
        hierarchical_MAP["Beta1"][i] - unpooled_MAP["Beta1"][i],
        fc="k",
        ec="k",
        length_includes_head=True,
        alpha=0.2,
        linewidth=1,
        head_width=0.04,
    )
ax.legend();

> **Exercise 30-5:** Examine the foregoing plot. The intercept value is on the horizontal axis and the floor (slope) is on the vertical axis. The blue dots show the parameter estimates for the unpooled model. The red dots show the coefficient values for the hierarchical model. The arrows connect the MAP coefficient values from the two models for the same county. Answer the following questions:          
> 1. What does the wide dispersion of the MAP values of the unpooled model coefficients tell you about the fit of this model give the exchangability of the observations?     
> 2. What do the closely packed coefficient MAP values of the hierarchical model tell you about the shrinkage of the coefficient values?
> 3. Given your observations about shrinkage, what can you say about any need to regularize the hierarchical Bayesian model?      

> **Answers:**      
> 1.              
> 2.               
> 3.          

#### Copyright 2022, 2023, 2024 Stephen F Elston. All rights reserved.   